In [ ]:
# majority of the code is taken from https://github.com/facebookresearch/head-to-tail/blob/main/head_to_tail_dbpedia.py
! git clone https://github.com/facebookresearch/head-to-tail

In [ ]:
#download manually DBPEdia dump from https://databus.dbpedia.org/dbpedia/mappings/mappingbased-objects

### Creating questions from templates using selected triples (with split by entity popularity)

In [1]:
import argparse
import bz2
from tqdm import tqdm
import json
import os
import requests
import pandas as pd
import random
from urllib.parse import unquote
import time

In [2]:
kg_dir =  "./"

In [3]:
entity = {}
relation = {}
with bz2.open(
    os.path.join(kg_dir, "mappingbased-objects_lang=en.ttl.bz2"), mode="rt"
) as f:
    for line in tqdm(f):
        line = line.strip().strip(".")
        cnt = 0
        block = 0
        start = 0
        for i in range(len(line)):
            if line[i] == "<":
                cnt += 1
            elif line[i] == ">":
                cnt -= 1
                if cnt == 0:
                    x = line[start : i + 1].strip()
                    start = i + 1
                    if block == 0 or block == 2:
                        if x not in entity:
                            entity[x] = 0
                        entity[x] += 1
                    else:
                        if x not in relation:
                            relation[x] = 0
                        relation[x] += 1
                    block += 1

22791171it [42:57, 8842.81it/s] 


In [ ]:
with open("entity_dump.txt", "w") as fp:
    json.dump(entity, fp)
    
with open("relations_dump.txt", "w") as fp:
    json.dump(relation, fp)

In [3]:
# with open("entities_dump.txt", "r", encoding="utf8") as f:
#     entity = json.load(f)

In [4]:
# with open("relations_dump.txt", "r", encoding="utf8") as f:
#     relation = json.load(f)

In [5]:
len(entity)

7958883

In [6]:
len(relation)

633

In [5]:
relation_sorted = [[x, relation[x]] for x in relation]

In [6]:
relation_sorted.sort(key=lambda x: -x[1])

In [7]:
entity_sorted = [[x, entity[x]] for x in entity]
entity_sorted.sort(key=lambda x: -x[1])

In [8]:
dbpedia_entity_htt_l = []
for x in entity_sorted:
    dbpedia_entity_htt_l += [x[1]]
dbpedia_entity_htt_l.sort()
dbpedia_entity_htt_l_s = [0]
for i in range(len(dbpedia_entity_htt_l)):
    dbpedia_entity_htt_l_s += [dbpedia_entity_htt_l_s[i] + dbpedia_entity_htt_l[i]]
dbpedia_entity_co1 = None
dbpedia_entity_co2 = None
for i in range(len(dbpedia_entity_htt_l)):
    if (
        dbpedia_entity_co1 is None
        and (dbpedia_entity_htt_l_s[i + 1] - dbpedia_entity_htt_l_s[0])
        / (
            dbpedia_entity_htt_l_s[len(dbpedia_entity_htt_l)]
            - dbpedia_entity_htt_l_s[0]
        )
        >= 1.0 / 3.0
    ):
        dbpedia_entity_co1 = i
    if (
        dbpedia_entity_co2 is None
        and (dbpedia_entity_htt_l_s[i + 1] - dbpedia_entity_htt_l_s[0])
        / (
            dbpedia_entity_htt_l_s[len(dbpedia_entity_htt_l)]
            - dbpedia_entity_htt_l_s[0]
        )
        >= 2.0 / 3.0
    ):
        dbpedia_entity_co2 = i

In [19]:
relation_htt_stat = {"head": {}, "torso": {}, "tail": {}}
with bz2.open(
    os.path.join(kg_dir, "mappingbased-objects_lang=en.ttl.bz2"), mode="rt"
) as f:
    for line in f:
        line = line.strip().strip(".")
        cnt = 0
        start = 0
        triple = []
        for i in range(len(line)):
            if line[i] == "<":
                cnt += 1
            elif line[i] == ">":
                cnt -= 1
                if cnt == 0:
                    x = line[start : i + 1].strip()
                    start = i + 1
                    triple += [x]
        if entity[triple[0]] <= dbpedia_entity_htt_l[dbpedia_entity_co1]:
            htt = "tail"
        elif entity[triple[0]] <= dbpedia_entity_htt_l[dbpedia_entity_co2]:
            htt = "torso"
        else:
            htt = "head"
        if triple[1] not in relation_htt_stat[htt]:
            relation_htt_stat[htt][triple[1]] = 0
        relation_htt_stat[htt][triple[1]] += 1

In [ ]:
with open("relation_htt_stat.txt", "w") as fp:
    json.dump(relation, fp)

with open("relation_htt_stat.txt", "r", encoding="utf8") as f:
    relation_htt_stat = json.load(f)

In [23]:
relation_htt_stat

{'head': {'<http://dbpedia.org/ontology/birthPlace>': 51724,
  '<http://dbpedia.org/ontology/genre>': 27234,
  '<http://dbpedia.org/ontology/hometown>': 5223,
  '<http://dbpedia.org/ontology/recordLabel>': 19041,
  '<http://www.w3.org/2000/01/rdf-schema#seeAlso>': 48937,
  '<http://xmlns.com/foaf/0.1/homepage>': 18863,
  '<http://dbpedia.org/ontology/author>': 810,
  '<http://dbpedia.org/ontology/company>': 1654,
  '<http://dbpedia.org/ontology/creator>': 952,
  '<http://dbpedia.org/ontology/director>': 1150,
  '<http://dbpedia.org/ontology/distributor>': 934,
  '<http://dbpedia.org/ontology/network>': 960,
  '<http://dbpedia.org/ontology/producer>': 944,
  '<http://dbpedia.org/ontology/starring>': 10520,
  '<http://dbpedia.org/ontology/country>': 16928,
  '<http://dbpedia.org/ontology/governingBody>': 865,
  '<http://dbpedia.org/ontology/politicalLeader>': 15734,
  '<http://dbpedia.org/ontology/subdivision>': 34353,
  '<http://dbpedia.org/ontology/timeZone>': 12824,
  '<http://dbpedia

In [10]:
with open("head-to-tail/dbpedia_question_templates.json", "r", encoding="utf8") as f:
    dbpedia_template = json.load(f)

In [11]:
dbpedia_template

{'<http://dbpedia.org/ontology/team>': 'Which team is _ a part of?',
 '<http://dbpedia.org/ontology/birthPlace>': '_ was born in which place?',
 '<http://dbpedia.org/ontology/subdivision>': 'What subdivision does _ belong to?',
 '<http://dbpedia.org/ontology/genre>': 'What genre does _ belong to?',
 '<http://dbpedia.org/ontology/recordLabel>': 'Which record label produced _?',
 '<http://xmlns.com/foaf/0.1/homepage>': 'What is the homepage of _?',
 '<http://dbpedia.org/ontology/country>': 'Which country is _ from?',
 '<http://dbpedia.org/ontology/occupation>': 'What is the occupation of _?',
 '<http://dbpedia.org/ontology/commander>': 'Who was the commander of _?',
 '<http://dbpedia.org/ontology/timeZone>': 'What is the time zone of _?',
 '<http://dbpedia.org/ontology/ground>': "What is the stadium where _'s home matches are held?",
 '<http://dbpedia.org/ontology/starring>': 'Who starred in _?',
 '<http://dbpedia.org/ontology/deathPlace>': 'Where did _ pass away?',
 '<http://dbpedia.org

In [12]:
len(dbpedia_template)

435

In [ ]:
#stop if the kernel is dead, the amount of collected triples will be enough

In [ ]:
random.seed(42)
dbpediagen = {"head": [], "torso": [], "tail": []}
current = [None, None]
records = []
currenthtt = None
with bz2.open(os.path.join(kg_dir, "mappingbased-objects_lang=en.ttl.bz2"), mode="rt") as f:

    for line in tqdm(f):
        line = line.strip().strip(".")
        cnt = 0
        start = 0
        triple = []
        for i in range(len(line)):
            if line[i] == "<":
                cnt += 1
            elif line[i] == ">":
                cnt -= 1
                if cnt == 0:
                    x = line[start : i + 1].strip()
                    start = i + 1
                    triple += [x]
        if entity[triple[0]] <= dbpedia_entity_htt_l[dbpedia_entity_co1]:
            htt = "tail"
        elif entity[triple[0]] <= dbpedia_entity_htt_l[dbpedia_entity_co2]:
            htt = "torso"
        else:
            htt = "head"
        if triple[:2] != current:
            if len(records) == 1:
                if current[1] in dbpedia_template:
                    r = random.random()
                    if r < 10000 / relation_htt_stat[currenthtt][current[1]]:
                        A = unquote(records[0][0].split("/")[-1][:-1])
                        B = unquote(
                            records[0][2].split("dbpedia.org/resource/")[-1][:-1]
                        )
                        if "__" not in A and "__" not in B:
                            A = A.replace("_", " ")
                            B = B.replace("_", " ")

                            if B.lower() not in A.lower():
                                q = dbpedia_template[current[1]].replace("_", A)
                                a = B
                                dbpediagen[currenthtt] += [
                                    [records[0], current[1], q, a]
                                ]
            current = triple[:2]
            records = [triple]
            currenthtt = htt
        else:
            records += [triple]

In [14]:
len(dbpediagen['head'])

43984

In [15]:
len(dbpediagen['torso'])

272114

In [16]:
len(dbpediagen['tail'])

350576

In [17]:
def cleaning_rules(rel, ans):
        if rel == "<http://dbpedia.org/ontology/category>":
            return True
        if rel == "<http://dbpedia.org/ontology/isPartOf>":
            return True
        if type(ans) == str:
            if rel == "<http://dbpedia.org/ontology/computingPlatform>" and ans == "Cross-platform":
                return True
            if rel == "<http://dbpedia.org/ontology/typeOfElectrification>" and ans in ["Volts", "Volt"]:
                return True
            if rel == "<http://dbpedia.org/ontology/militaryRank>" and ans.startswith("Military ranks "):
                return True
            if rel == "<http://dbpedia.org/ontology/operatingSystem>" and ans.endswith("system software"):
                return True
            if rel == "<http://dbpedia.org/ontology/deathCause>" and ans == "Death":
                return True
            if rel == "<http://dbpedia.org/ontology/deathCause>" and ans.startswith("Death of "):
                return True
            if rel == "<http://dbpedia.org/ontology/deathCause>" and ans.startswith("Killing of "):
                return True
            if rel == "<http://dbpedia.org/ontology/notableCommander>" and ans in ["Major general", "General"]:
                return True
            if rel == "<http://dbpedia.org/ontology/ethnicGroup>" and ans.startswith("Ethnic groups "):
                return True
            if rel == "<http://dbpedia.org/ontology/island>" and ans == "Islands":
                return True
            if rel == "<http://dbpedia.org/ontology/presenter>" and ans.endswith("Awards"):
                return True
            if rel == "<http://dbpedia.org/ontology/symptom>" and ans == "Symptoms":
                return True
            if rel == "<http://dbpedia.org/ontology/honours>" and ans.startswith("Honours of "):
                return True
            if rel == "<http://dbpedia.org/ontology/canton>" and ans.startswith("Cantons of "):
                return True
            if rel == "<http://dbpedia.org/ontology/battery>" and ans in ["KWh", "Kilowatt-hour"]:
                return True
            if rel == "<http://dbpedia.org/ontology/college>" and ans.endswith("basketball"):
                return True
            if ans.startswith("List of "):
                return True
        return False

In [20]:
output = {"head": [], "torso": [], "tail": []}
maxsample = {}
for x in ["head", "torso", "tail"]:
    for i in range(len(dbpediagen[x])):
        if type(dbpediagen[x][i][3]) == str and dbpediagen[x][i][3].startswith("<http"):
            if dbpediagen[x][i][1] not in [
                "<http://xmlns.com/foaf/0.1/page>",
                "<http://xmlns.com/foaf/0.1/homepage>",
                "<http://dbpedia.org/ontology/webcast>",
            ]:
                continue
            dbpediagen[x][i][3] = dbpediagen[x][i][3][1:]
        if cleaning_rules(dbpediagen[x][i][1], dbpediagen[x][i][3]):
            continue
        if "<http://dbpedia.org/resource/List_of_" in dbpediagen[x][i][0][0]:
            continue
        if dbpediagen[x][i][1] not in maxsample:
            maxsample[dbpediagen[x][i][1]] = {}
        if x not in maxsample[dbpediagen[x][i][1]]:
            maxsample[dbpediagen[x][i][1]][x] = 0
        maxsample[dbpediagen[x][i][1]][x] += 1
for x in maxsample:
    maxsample[x] = min(
        maxsample[x]["head"] if "head" in maxsample[x] else 0,
        maxsample[x]["torso"] if "torso" in maxsample[x] else 0,
        maxsample[x]["tail"] if "tail" in maxsample[x] else 0,
    )

In [21]:
for x in ["tail", "torso", "head"]:
    stat = {}
    random.shuffle(dbpediagen[x])
    for i in range(len(dbpediagen[x])):
        if type(dbpediagen[x][i][3]) == str and dbpediagen[x][i][3].startswith("<http"):
            if dbpediagen[x][i][1] not in [
                "<http://xmlns.com/foaf/0.1/page>",
                "<http://xmlns.com/foaf/0.1/homepage>",
                "<http://dbpedia.org/ontology/webcast>",
            ]:
                continue
            dbpediagen[x][i][3] = dbpediagen[x][i][3][1:]
        if cleaning_rules(dbpediagen[x][i][1], dbpediagen[x][i][3]):
            continue
        if "<http://dbpedia.org/resource/List_of_" in dbpediagen[x][i][0][0]:
            continue
        if dbpediagen[x][i][1] not in stat:
            stat[dbpediagen[x][i][1]] = 0
        if (
            stat[dbpediagen[x][i][1]] < 30 #increase this parameter to increase amount of collected triples
            and stat[dbpediagen[x][i][1]] < maxsample[dbpediagen[x][i][1]]
            and dbpediagen[x][i][2] != ""
            and dbpediagen[x][i][3].lower() not in dbpediagen[x][i][2].lower()
        ):
            output[x] += [dbpediagen[x][i]]
            stat[dbpediagen[x][i][1]] += 1
    print(x, sum([1 for a in stat if stat[a] > 0]), sum([stat[a] for a in stat]))

tail 344 5850
torso 344 5850
head 344 5850


In [22]:
for x in ["head", "torso", "tail"]:
        for i in range(len(output[x])):
            output[x][i][2] = (
                output[x][i][2].replace(" the the ", " the ").replace(" the The ", " the ")
            )
            if output[x][i][2].endswith(")?"):
                a = output[x][i][2].split("(")
                if len(a) > 1:
                    l = "(".join(a[:-1])
                    r = a[-1][:-2]
                    if " " + r.lower() + " " in l.lower():
                        output[x][i][2] = l + "?"

In [23]:
 for x in ["head", "torso", "tail"]:
        random.shuffle(output[x])

### Save the collected file

In [27]:
with open(os.path.join("./", "head_to_tail_dbpedia_17550.json"), "w", encoding="utf8") as f:
        json.dump(output, f, indent=1, ensure_ascii=False)

### Reorganise the collected file

In [51]:
f = open( "head_to_tail_dbpedia_17550.json")
data = json.load(f)

In [52]:
data_flatened = pd.DataFrame(columns=['subject','relation','object','template',
                                      'question','popularity_range', 'answer'])

In [53]:
len(data['head'])

5850

In [54]:
for i in range(len(data['head'])):
    
    data_flatened.loc[i, 'subject'] = data['head'][i][0][0]
    data_flatened.loc[i, 'relation'] = data['head'][i][0][1]
    data_flatened.loc[i, 'object'] = data['head'][i][0][2]
    data_flatened.loc[i, 'template'] = data['head'][i][1]
    data_flatened.loc[i, 'question'] =  data['head'][i][2]
    data_flatened.loc[i, 'popularity_range'] =  'head'
    data_flatened.loc[i, 'answer'] =  data['head'][i][3]

In [55]:
len(data['torso'])

5850

In [56]:
for i in range(5850, 5850+len(data['tail'])):
    
    data_flatened.loc[i, 'subject'] = data['torso'][i-5850][0][0]
    data_flatened.loc[i, 'relation'] = data['torso'][i-5850][0][1]
    data_flatened.loc[i, 'object'] = data['torso'][i-5850][0][2]
    data_flatened.loc[i, 'template'] = data['torso'][i-5850][1]
    data_flatened.loc[i, 'question'] =  data['torso'][i-5850][2]
    data_flatened.loc[i, 'popularity_range'] =  'torso'
    data_flatened.loc[i, 'answer'] =  data['torso'][i-5850][3]

In [57]:
for i in range(5850*2, 5850*2+len(data['tail'])):
    
    data_flatened.loc[i, 'subject'] = data['tail'][i-11700][0][0]
    data_flatened.loc[i, 'relation'] = data['tail'][i-11700][0][1]
    data_flatened.loc[i, 'object'] = data['tail'][i-11700][0][2]
    data_flatened.loc[i, 'template'] = data['tail'][i-11700][1]
    data_flatened.loc[i, 'question'] =  data['tail'][i-11700][2]
    data_flatened.loc[i, 'popularity_range'] =  'tail'
    data_flatened.loc[i, 'answer'] =  data['tail'][i-11700][3]

In [18]:
data_flatened.to_csv('data_flatened_dbpedia_by_popularity_17550.csv', index = False)

### Function that finds Wikidata ID by DBPedia ID

In [42]:
def dbpedia_to_wikidata(entity):
    
    url = "https://dbpedia.org/sparql"
    query = """
         PREFIX dbr: <http://dbpedia.org/resource/>
         SELECT distinct ?wikidata_concept
         WHERE {<ENTITY> owl:sameAs ?wikidata_concept
         FILTER(regex(str(?wikidata_concept), "www.wikidata.org" ) )}
                """.replace(
                    "<ENTITY>", entity)
    request = requests.get(url, params={"format": "json", "query": query})
    data = request.json()
    
    return data['results']['bindings'][0]['wikidata_concept']['value']

In [ ]:
#collect wikidata IDs

In [69]:
for i in tqdm(range(len(data_flatened))):
    try:
        if pd.isna(data_flatened.loc[i, 'object_wikidata_id']):
            cur_ent = data_flatened.loc[i, 'object']
            data_flatened.loc[i, 'object_wikidata_id'] = dbpedia_to_wikidata(cur_ent)
            time.sleep(1)
    except:
        continue

100%|██████████| 17550/17550 [26:06<00:00, 11.20it/s]   


In [71]:
len(data_flatened.query('object_wikidata_id.isna()'))

3078

In [72]:
data_flatened.to_csv('data_flatened_dbpedia_by_popularity_17550.csv', index = False)

### Function that finds Wikidata aliases by Wikidata ID

In [73]:
def return_entity_aliases(entity_id):
    
    url = "https://query.wikidata.org/sparql"
    query = """
          PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?alias WHERE {
  <ENTITY> (skos:altLabel|rdfs:label) ?alias .
  FILTER (LANG(?alias) = "en")
}
                """.replace(
                    "<ENTITY>", entity_id)
    request = requests.get(url, params={"format": "json", "query": query})
    data = request.json()
    aliases = []
    for j in range(len(data['results']['bindings'])):
        aliases.append(data['results']['bindings'][j]['alias']['value'])
    return aliases

In [94]:
for i in tqdm(range(len(data_flatened))):
    try:
         if pd.isna(data_flatened.loc[i, 'object_wikidata_aliases']):
            cur_id = data_flatened.loc[i, 'object_wikidata_id']
            data_flatened.loc[i, 'object_wikidata_aliases'] = ', '.join(return_entity_aliases('<'+cur_id+'>'))
            time.sleep(2)
    except:
        continue

100%|██████████| 17550/17550 [00:00<00:00, 34458.63it/s]


In [96]:
data_flatened.to_csv('data_flatened_dbpedia_by_popularity_17550.csv', index = False)

In [95]:
len(data_flatened.query('object_wikidata_aliases.isna()'))

3078

In [87]:
len(data_flatened.query('object_wikidata_id.isna()'))

3078